In [ ]:
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()

In [ ]:
import ee
import os
import glob
import json
import copy
import sys
import time

In [ ]:
dict_expression = {
    'ratio_median': "float(b('nir_median') / b('red_median'))",
    'ratio_median_wet': "float(b('nir_median_wet') / b('red_median_wet'))",
    'ratio_median_dry': "float(b('nir_median_dry') / b('red_median_dry'))",
    'rvi_median': "float(b('red_median') / b('nir_median'))",
    'rvi_median_wet': "float(b('red_median_wet') / b('nir_median_wet'))",
    'rvi_median_dry': "float(b('red_median_dry') / b('nir_median_dry'))",
    'ndvi_median': "float(b('nir_median') - b('red_median')) / (b('nir_median') + b('red_median'))",
    'ndvi_median_wet': "float(b('nir_median_wet') - b('red_median_wet')) / (b('nir_median_wet') + b('red_median_wet'))",
    'ndvi_median_dry': "float(b('nir_median_dry') - b('red_median_dry')) / (b('nir_median_dry') + b('red_median_dry'))",
    'ndbi_median': "float(b('swir1_median') - b('nir_median')) / (b('swir1_median') + b('nir_median'))",
    'ndbi_median_wet': "float(b('swir1_median_wet') - b('nir_median_wet')) / (b('swir1_median_wet') + b('nir_median_wet'))",
    'ndbi_median_dry': "float(b('swir1_median_dry') - b('nir_median_dry')) / (b('swir1_median_dry') + b('nir_median_dry'))",
    'ndmi_median': "float(b('nir_median') - b('swir1_median')) / (b('nir_median') + b('swir1_median'))",
    'ndmi_median_wet': "float(b('nir_median_wet') - b('swir1_median_wet')) / (b('nir_median_wet') + b('swir1_median_wet'))",
    'ndmi_median_dry': "float(b('nir_median_dry') - b('swir1_median_dry')) / (b('nir_median_dry') + b('swir1_median_dry'))",
    'nbr_median': "float(b('nir_median') - b('swir1_median')) / (b('nir_median') + b('swir1_median'))",
    'nbr_median_wet': "float(b('nir_median_wet') - b('swir1_median_wet')) / (b('nir_median_wet') + b('swir1_median_wet'))",
    'nbr_median_dry': "float(b('nir_median_dry') - b('swir1_median_dry')) / (b('nir_median_dry') + b('swir1_median_dry'))",
    'ndti_median': "float(b('swir1_median') - b('swir2_median')) / (b('swir1_median') + b('swir2_median'))",
    'ndti_median_wet': "float(b('swir1_median_wet') - b('swir2_median_wet')) / (b('swir1_median_wet') + b('swir2_median_wet'))",
    'ndti_median_dry': "float(b('swir1_median_dry') - b('swir2_median_dry')) / (b('swir1_median_dry') + b('swir2_median_dry'))",
    'ndwi_median': "float(b('nir_median') - b('swir2_median')) / (b('nir_median') + b('swir2_median'))",
    'ndwi_median_wet': "float(b('nir_median_wet') - b('swir2_median_wet')) / (b('nir_median_wet') + b('swir2_median_wet'))",
    'ndwi_median_dry': "float(b('nir_median_dry') - b('swir2_median_dry')) / (b('nir_median_dry') + b('swir2_median_dry'))",
    "awei_median": "float(4 * (b('green_median') - b('swir2_median')) - (0.25 * b('nir_median') + 2.75 * b('swir1_median')))",
    "awei_median_wet": "float(4 * (b('green_median_wet') - b('swir2_median_wet')) - (0.25 * b('nir_median_wet') + 2.75 * b('swir1_median_wet')))",
    "awei_median_dry": "float(4 * (b('green_median_dry') - b('swir2_median_dry')) - (0.25 * b('nir_median_dry') + 2.75 * b('swir1_median_dry')))",
    "iia_median": "float((b('green_median') - 4 *  b('nir_median')) / (b('green_median') + 4 *  b('nir_median')))",
    "iia_median_wet": "float((b('green_median_wet') - 4 *  b('nir_median_wet')) / (b('green_median_wet') + 4 *  b('nir_median_wet')))",
    "iia_median_dry": "float((b('green_median_dry') - 4 *  b('nir_median_dry')) / (b('green_median_dry') + 4 *  b('nir_median_dry')))",
    'evi_median': "float(2.4 * (b('nir_median') - b('red_median')) / (1 + b('nir_median') + b('red_median')))",
    'evi_median_wet': "float(2.4 * (b('nir_median_wet') - b('red_median_wet')) / (1 + b('nir_median_wet') + b('red_median_wet')))",
    'evi_median_dry': "float(2.4 * (b('nir_median_dry') - b('red_median_dry')) / (1 + b('nir_median_dry') + b('red_median_dry')))",
    'gvmi_median': "float ((b('nir_median')  + 0.1) - (b('swir1_median') + 0.02)) / ((b('nir_median') + 0.1) + (b('swir1_median') + 0.02))",
    'gvmi_median_wet': "float ((b('nir_median_wet')  + 0.1) - (b('swir1_median_wet') + 0.02)) / ((b('nir_median_wet') + 0.1) + (b('swir1_median_wet') + 0.02))",
    'gvmi_median_dry': "float ((b('nir_median_dry')  + 0.1) - (b('swir1_median_dry') + 0.02)) / ((b('nir_median_dry') + 0.1) + (b('swir1_median_dry') + 0.02))",
    'gcvi_median': "float(b('nir_median')) / (b('green_median')) - 1",
    'gcvi_median_wet': "float(b('nir_median_wet')) / (b('green_median_wet')) - 1",
    'gcvi_median_dry': "float(b('nir_median_dry')) / (b('green_median_dry')) - 1",
    'gemi_median': "float((2 * (b('nir_median') * b('nir_median') - b('red_median') * b('red_median')) + 1.5 * b('nir_median') + 0.5 * b('red_median')) / (b('nir_median') + b('green_median') + 0.5) )",
    'gemi_median_wet': "float((2 * (b('nir_median_wet') * b('nir_median_wet') - b('red_median_wet') * b('red_median_wet')) + 1.5 * b('nir_median_wet') + 0.5 * b('red_median_wet')) / (b('nir_median_wet') + b('green_median_wet') + 0.5) )",
    'gemi_median_dry':  "float((2 * (b('nir_median_dry') * b('nir_median_dry') - b('red_median_dry') * b('red_median_dry')) + 1.5 * b('nir_median_dry') + 0.5 * b('red_median_dry')) / (b('nir_median_dry') + b('green_median_dry') + 0.5) )",
    'cvi_median': "float(b('nir_median') * (b('green_median') / (b('blue_median') * b('blue_median'))))",
    'cvi_median_wet': "float(b('nir_median_wet') * (b('green_median_wet') / (b('blue_median_wet') * b('blue_median_wet'))))",
    'cvi_median_dry': "float(b('nir_median_dry') * (b('green_median_dry') / (b('blue_median_dry') * b('blue_median_dry'))))",
    'gli_median': "float((2 * b('green_median') - b('red_median') - b('blue_median')) / (2 * b('green_median') + b('red_median') + b('blue_median')))",
    'gli_median_wet': "float((2 * b('green_median_wet') - b('red_median_wet') - b('blue_median_wet')) / (2 * b('green_median_wet') + b('red_median_wet') + b('blue_median_wet')))",
    'gli_median_dry': "float((2 * b('green_median_dry') - b('red_median_dry') - b('blue_median_dry')) / (2 * b('green_median_dry') + b('red_median_dry') + b('blue_median_dry')))",
    'shape_median': "float((2 * b('red_median') - b('green_median') - b('blue_median')) / (b('green_median') - b('blue_median')))",
    'shape_median_wet': "float((2 * b('red_median_wet') - b('green_median_wet') - b('blue_median_wet')) / (b('green_median_wet') - b('blue_median_wet')))",
    'shape_median_dry': "float((2 * b('red_median_dry') - b('green_median_dry') - b('blue_median_dry')) / (b('green_median_dry') - b('blue_median_dry')))",
    'afvi_median': "float((b('nir_median') - 0.5 * b('swir2_median')) / (b('nir_median') + 0.5 * b('swir2_median')))",
    'afvi_median_wet': "float((b('nir_median_wet') - 0.5 * b('swir2_median_wet')) / (b('nir_median_wet') + 0.5 * b('swir2_median_wet')))",
    'afvi_median_dry': "float((b('nir_median_dry') - 0.5 * b('swir2_median_dry')) / (b('nir_median_dry') + 0.5 * b('swir2_median_dry')))",
    'avi_median': "float((b('nir_median')* (1.0 - b('red_median')) * (b('nir_median') - b('red_median'))) ** 1/3)",
    'avi_median_wet': "float((b('nir_median_wet')* (1.0 - b('red_median_wet')) * (b('nir_median_wet') - b('red_median_wet'))) ** 1/3)",
    'avi_median_dry': "float((b('nir_median_dry')* (1.0 - b('red_median_dry')) * (b('nir_median_dry') - b('red_median_dry'))) ** 1/3)",
    'nddi_median': "float((b('ndvi_median') - b('ndwi_median')) / (b('ndvi_median') + b('ndwi_median')))",
    'nddi_median_wet': "float((b('ndvi_median_wet') - b('ndwi_median_wet')) / (b('ndvi_median_wet') + b('ndwi_median_wet')))",
    'nddi_median_dry': "float((b('ndvi_median_dry') - b('ndwi_median_dry')) / (b('ndvi_median_dry') + b('ndwi_median_dry')))",
    'bsi_median': "float(((b('swir1_median') - b('red_median')) - (b('nir_median') + b('blue_median'))) / ((b('swir1_median') + b('red_median')) + (b('nir_median') + b('blue_median'))))",
    'bsi_median_wet': "float(((b('swir1_median') - b('red_median')) - (b('nir_median') + b('blue_median'))) / ((b('swir1_median') + b('red_median')) + (b('nir_median') + b('blue_median'))))",
    'bsi_median_dry': "float(((b('swir1_median') - b('red_median')) - (b('nir_median') + b('blue_median'))) / ((b('swir1_median') + b('red_median')) + (b('nir_median') + b('blue_median'))))",
    'brba_median': "float(b('red_median') / b('swir1_median'))",
    'brba_median_wet': "float(b('red_median_wet') / b('swir1_median_wet'))",
    'brba_median_dry': "float(b('red_median_dry') / b('swir1_median_dry'))",
    'dswi5_median': "float((b('nir_median') + b('green_median')) / (b('swir1_median') + b('red_median')))",
    'dswi5_median_wet': "float((b('nir_median_wet') + b('green_median_wet')) / (b('swir1_median_wet') + b('red_median_wet')))",
    'dswi5_median_dry': "float((b('nir_median_dry') + b('green_median_dry')) / (b('swir1_median_dry') + b('red_median_dry')))",
    'lswi_median': "float((b('nir_median') - b('swir1_median')) / (b('nir_median') + b('swir1_median')))",
    'lswi_median_wet': "float((b('nir_median_wet') - b('swir1_median_wet')) / (b('nir_median_wet') + b('swir1_median_wet')))",
    'lswi_median_dry': "float((b('nir_median_dry') - b('swir1_median_dry')) / (b('nir_median_dry') + b('swir1_median_dry')))",
    'mbi_median': "float(((b('swir1_median') - b('swir2_median') - b('nir_median')) / (b('swir1_median') + b('swir2_median') + b('nir_median'))) + 0.5)",
    'mbi_median_wet': "float(((b('swir1_median_wet') - b('swir2_median_wet') - b('nir_median_wet')) / (b('swir1_median_wet') + b('swir2_median_wet') + b('nir_median_wet'))) + 0.5)",
    'mbi_median_dry': "float(((b('swir1_median_dry') - b('swir2_median_dry') - b('nir_median_dry')) / (b('swir1_median_dry') + b('swir2_median_dry') + b('nir_median_dry'))) + 0.5)",
    'ui_median': "float((b('swir2_median') - b('nir_median')) / (b('swir2_median') + b('nir_median')))",
    'ui_median_wet': "float((b('swir2_median_wet') - b('nir_median_wet')) / (b('swir2_median_wet') + b('nir_median_wet')))",
    'ui_median_dry': "float((b('swir2_median_dry') - b('nir_median_dry')) / (b('swir2_median_dry') + b('nir_median_dry')))",
    'osavi_median': "float(b('nir_median') - b('red_median')) / (0.16 + b('nir_median') + b('red_median'))",
    'osavi_median_wet': "float(b('nir_median_wet') - b('red_median_wet')) / (0.16 + b('nir_median_wet') + b('red_median_wet'))",
    'osavi_median_dry': "float(b('nir_median_dry') - b('red_median_dry')) / (0.16 + b('nir_median_dry') + b('red_median_dry'))",
    'msavi_median': "float((2 * b('nir_median') + 1 - sqrt((2 * b('nir_median') + 1) * (2 * b('nir_median') + 1) - 8 * (b('nir_median') - b('red_median'))))/2)",
    'msavi_median_wet': "float((2 * b('nir_median_wet') + 1 - sqrt((2 * b('nir_median_wet') + 1) * (2 * b('nir_median_wet') + 1) - 8 * (b('nir_median_wet') - b('red_median_wet'))))/2)",
    'msavi_median_dry': "float((2 * b('nir_median_dry') + 1 - sqrt((2 * b('nir_median_dry') + 1) * (2 * b('nir_median_dry') + 1) - 8 * (b('nir_median_dry') - b('red_median_dry'))))/2)",
    'gsavi_median': "float(b('nir_median') - b('green_median')) / ((0.5 + b('nir_median') + b('green_median')) * 1.5)",
    'gsavi_median_wet': "float(b('nir_median_wet') - b('green_median_wet')) / ((0.5 + b('nir_median_wet') + b('green_median_wet')) * 1.5)",
    'gsavi_median_dry': "float(b('nir_median_dry') - b('green_median_dry')) / ((0.5 + b('nir_median_dry') + b('green_median_dry')) * 1.5)",
    'ri_median': "float(b('nir_median') - b('green_median')) / (b('nir_median') + b('green_median'))",
    'ri_median_wet': "float(b('nir_median_wet') - b('green_median_wet')) / (b('nir_median_wet') + b('green_median_wet'))",
    'ri_median_dry': "float(b('nir_median_dry') - b('green_median_dry')) / (b('nir_median_dry') + b('green_median_dry'))",
    'brightness_median': "float(0.3037 * b('blue_median') + 0.2793 * b('green_median') + 0.4743 * b('red_median')  + 0.5585 * b('nir_median') + 0.5082 * b('swir1_median') +  0.1863 * b('swir2_median'))",
    'brightness_median_wet': "float(0.3037 * b('blue_median_wet') + 0.2793 * b('green_median_wet') + 0.4743 * b('red_median_wet') + 0.5585 * b('nir_median_wet') + 0.5082 * b('swir1_median_wet') +  0.1863 * b('swir2_median_wet'))",
    'brightness_median_dry': "float(0.3037 * b('blue_median_dry') + 0.2793 * b('green_median_dry') + 0.4743 * b('red_median_dry') + 0.5585 * b('nir_median_dry') + 0.5082 * b('swir1_median_dry') +  0.1863 * b('swir2_median_dry'))",
    'wetness_median': "float(0.1509 * b('blue_median') + 0.1973 * b('green_median') + 0.3279 * b('red_median')  + 0.3406 * b('nir_median') + 0.7112 * b('swir1_median') +  0.4572 * b('swir2_median'))",
    'wetness_median_wet': "float(0.1509 * b('blue_median_wet') + 0.1973 * b('green_median_wet') + 0.3279 * b('red_median_wet') + 0.3406 * b('nir_median_wet') + 0.7112 * b('swir1_median_wet') +  0.4572 * b('swir2_median_wet'))",
    'wetness_median_dry': "float(0.1509 * b('blue_median_dry') + 0.1973 * b('green_median_dry') + 0.3279 * b('red_median_dry') + 0.3406 * b('nir_median_dry') + 0.7112 * b('swir1_median_dry') +  0.4572 * b('swir2_median_dry'))",
    'msi_median': "float( b('nir_median') / b('swir1_median'))",
    'msi_median_wet': "float( b('nir_median_wet') / b('swir1_median_wet'))",
    'msi_median_dry': "float( b('nir_median_dry') / b('swir1_median_dry'))",
    'spri_median': "float(((b('green_median') - b('blue_median')) / (b('green_median') + b('blue_median'))) + 1) / 2",
    'spri_median_wet': "float(((b('green_median_wet') - b('blue_median_wet')) / (b('green_median_wet') + b('blue_median_wet'))) + 1) / 2",
    'spri_median_dry': "float(((b('green_median') - b('blue_median')) / (b('green_median') + b('blue_median'))) + 1) / 2",
    'co2flux_median': "float(b('ndvi_median') * b('spri_median'))",
    'co2flux_median_wet': "float(b('ndvi_median_wet') * b('spri_median_wet'))",
    'co2flux_median_dry': "float(b('ndvi_median_dry') * b('spri_median_dry'))",
}

In [ ]:
projAccount = 'geo-data-s'
try:
    ee.Initialize(project= projAccount)
    print('The Earth Engine package initialized successfully!')
except ee.EEException as e:
    print('The Earth Engine package failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

The Earth Engine package initialized successfully!


In [ ]:
options = {
        'bnd_L': ['blue','green','red','nir','swir1','swir2'],
        'bnd_fraction': ['gv','npv','soil'],
        'biomas': ['CERRADO','CAATINGA','MATAATLANTICA'],
        'bioma': "CAATINGA",
        'version': 4,
        'lsBandasMap': [],
        'asset_bacias_buffer' : 'projects/mapbiomas-workspace/AMOSTRAS/col9/CAATINGA/bacias_hidrografica_caatinga_49_regions',
        'asset_grad': 'projects/mapbiomas-workspace/AMOSTRAS/col9/CAATINGA/basegrade30KMCaatinga',
        'assetMapbiomas90': 'projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_integration_v1',
        'asset_collectionId': 'LANDSAT/COMPOSITES/C02/T1_L2_32DAY',
        'asset_mosaic': 'projects/nexgenmap/MapBiomas2/LANDSAT/BRAZIL/mosaics-2',
        'asset_joinsGrBa': 'projects/mapbiomas-workspace/AMOSTRAS/col10/CAATINGA/ROIs/ROIs_cleaned_downsamplesv4C',
        'asset_joinsGrBaMB': 'projects/mapbiomas-workspace/AMOSTRAS/col10/CAATINGA/ROIs/ROIs_cleaned_MB_V4C',
        # 'asset_joinsGrBa': 'projects/mapbiomas-workspace/AMOSTRAS/col10/CAATINGA/ROIs/ROIs_cleaned_DS_v4corrCC',
        # 'asset_joinsGrBaMB': 'projects/mapbiomas-workspace/AMOSTRAS/col10/CAATINGA/ROIs/ROIs_cleaned_MB_DS_v4corrCC',
        # 'assetOutMB': 'projects/mapbiomas-workspace/AMOSTRAS/col10/CAATINGA/Classifier/Classify_fromMMBV2',
        'assetOutMB': 'projects/mapbiomas-workspace/AMOSTRAS/col10/CAATINGA/Classifier/Classify_fromMMBV2',
        'assetOut': 'projects/mapbiomas-workspace/AMOSTRAS/col10/CAATINGA/Classifier/ClassifyV2YY',
        # 'asset_output': 'projects/nexgenmap/SAMPLES/Caatinga',
        # Spectral bands selected
        "anoIntInit": 1985,
        "anoIntFin": 2024,
        # 'dict_classChangeBa': arqParams.dictClassRepre,
        # https://scikit-learn.org/stable/modules/ensemble.html#gradient-boosting
        'pmtGTB': {
            'numberOfTrees': 25,
            'shrinkage': 0.1,
            'samplingRate': 0.8,
            'loss': "LeastSquares",#'Huber',#'LeastAbsoluteDeviation',
            'seed': 0
        },
    }


In [ ]:
band_year = [nband + '_median' for nband in options['bnd_L']]
band_drys = [bnd + '_dry' for bnd in band_year]
band_wets = [bnd + '_wet' for bnd in band_year]
band_mosaic = band_year + band_wets + band_drys
print("lista de bandas do mosaico ")
print(band_mosaic)

lista de bandas do mosaico 
['blue_median', 'green_median', 'red_median', 'nir_median', 'swir1_median', 'swir2_median', 'blue_median_wet', 'green_median_wet', 'red_median_wet', 'nir_median_wet', 'swir1_median_wet', 'swir2_median_wet', 'blue_median_dry', 'green_median_dry', 'red_median_dry', 'nir_median_dry', 'swir1_median_dry', 'swir2_median_dry']


In [ ]:
# construção do mosaico Mapbiomas
lstSat = ["l5","l7","l8"]
imgMosaic = (
            ee.ImageCollection(options['asset_mosaic'])
                            .filter(ee.Filter.inList('biome', options['biomas']))
                            .filter(ee.Filter.inList('satellite', lstSat))
                            # .filterBounds(self.baciabuffer)
                            .select(band_mosaic)
        )
print(" tamanho de imagens ", imgMosaic.size().getInfo())


 tamanho de imagens  21946


In [ ]:
# carregando o mosaico de Google Earth
imagens_mosaicoEE = (
            ee.ImageCollection(options['asset_collectionId'])
                    # .filterBounds(self.baciabuffer)
                    .select(options['bnd_L'])
        )


In [ ]:
def aggregateBandsIndexTexture(img):
    # img = img.toInt()
    textura2 = img.select('nir_median').multiply(10000).toUint16().glcmTexture(3)
    contrastnir = textura2.select('nir_median_contrast').divide(10000).toFloat()
    textura2Dry = img.select('nir_median_dry').multiply(10000).toUint16().glcmTexture(3)
    contrastnirDry = textura2Dry.select('nir_median_dry_contrast').divide(10000).toFloat()
    #
    textura2R = img.select('red_median').multiply(10000).toUint16().glcmTexture(3)
    contrastred = textura2R.select('red_median_contrast').divide(10000).toFloat()
    textura2RDry = img.select('red_median_dry').multiply(10000).toUint16().glcmTexture(3)
    contrastredDry = textura2RDry.select('red_median_dry_contrast').divide(10000).toFloat()

    return img.addBands(contrastnir).addBands(contrastred).addBands(contrastnirDry).addBands(contrastredDry)

In [ ]:
def CalculateIndice(imagem, lstBandasSpectrals):
    # imageW = self.agregateBandswithSpectralIndex(imagem)
    # imageW = self.agregate_Bands_SMA_NDFIa(imageW)
    # imageW = self.addSlopeAndHilshade(imageW)

    imageW = aggregateBandsIndexTexture(imagem)
    lstBandsIndices = [kband  for kband in lstBandasSpectrals if kband not in band_mosaic and 'contrast' not in kband]
    # lstIndSpect = ee.List([])
    for bndSpect in lstBandsIndices:
        if 'nddi' in bndSpect:  #nddi_median_dry
            sufixo = bndSpect.replace('nddi_', '')
            expressao2 = dict_expression['ndvi_' + sufixo]
            pImgSpect2 = imageW.expression(expressao2).rename(['ndvi_' + sufixo])
            # imageW = imageW.addBands(pImgSpect2)

            expressao3 = dict_expression['ndwi_' + sufixo]
            pImgSpect3 = imageW.expression(expressao3).rename(['ndwi_' + sufixo])
            # imageW = imageW.addBands(pImgSpect3)

            expressao = dict_expression[bndSpect]
            pImgSpect = pImgSpect2.addBands(pImgSpect3).expression(expressao).rename([bndSpect])
            # lstIndSpect = lstIndSpect.add(pImgSpect)
            imageW = imageW.addBands(pImgSpect)

        elif 'co2flux' in bndSpect:
            sufixo = bndSpect.replace('co2flux_', '')
            expressao2 = dict_expression['ndvi_' + sufixo]
            pImgSpect2 = imageW.expression(expressao2).rename(['ndvi_' + sufixo])
            # imageW = imageW.addBands(pImgSpect2)

            expressao3 = dict_expression['spri_' + sufixo]
            pImgSpect3 = imageW.expression(expressao3).rename(['spri_' + sufixo])
            # imageW = imageW.addBands(pImgSpect3)

            expressao = dict_expression[bndSpect]
            pImgSpect = pImgSpect2.addBands(pImgSpect3).expression(expressao).rename([bndSpect])
            # lstIndSpect = lstIndSpect.add(pImgSpect)
            imageW = imageW.addBands(pImgSpect)

        else:
            expressao = dict_expression[bndSpect]
            # print(f" loading Spectral Band {bndSpect} >>> ") #{expressao}
            pImgSpect = imageW.expression(expressao).rename([bndSpect])
            # lstIndSpect = lstIndSpect.add(pImgSpect)
            imageW = imageW.addBands(pImgSpect)

    # imgInd = ee.Image.cat(lstIndSpect)
    return imageW

In [ ]:
def make_mosaicofromIntervalo(colMosaic, year_courrent):
    band_year = [nband + '_median' for nband in options['bnd_L']]
    band_drys = [bnd + '_dry' for bnd in band_year]
    band_wets = [bnd + '_wet' for bnd in band_year]

    dictPer = {
        'year': {
            'start': str(str(year_courrent)) + '-01-01',
            'end': str(year_courrent) + '-12-31',
            'surf': 'year',
            'bnds': band_year
        },
        'dry': {
            'start': str(year_courrent) + '-08-01',
            'end': str(year_courrent) + '-12-31',
            'surf': 'dry',
            'bnds': band_drys
        },
        'wet': {
            'start': str(year_courrent) + '-01-01',
            'end': str(year_courrent) + '-07-31',
            'surf': 'wet',
            'bnds': band_wets
        }
    }
    mosaico = None
    lstPeriodo = ['year', 'dry', 'wet']
    for periodo in lstPeriodo:
        dateStart =  dictPer[periodo]['start']
        dateEnd = dictPer[periodo]['end']
        bands_period = dictPer[periodo]['bnds']
        # get dry median mosaic
        mosaictmp = (
            colMosaic.select(options['bnd_L'])
                .filter(ee.Filter.date(dateStart, dateEnd))
                .median()
                .rename(bands_period)
        )
        if periodo == 'year':
            mosaico = copy.deepcopy(mosaictmp)
        else:
            mosaico = mosaico.addBands(mosaictmp)

    return mosaico

In [189]:
# definindo ano e bacia
nbacia = '7616'#'7544'
nyear = 2022

In [190]:
# loading geometry bacim
baciabuffer = ee.FeatureCollection(options['asset_bacias_buffer']).filter(
                    ee.Filter.eq('nunivotto4', nbacia))
print(f"know about the geometry 'nunivotto4' >>  {nbacia} loaded < {baciabuffer.size().getInfo()} > geometry" )
baciabuffer = baciabuffer.map(lambda f: f.set('id_codigo', 1))
print(baciabuffer.first().getInfo()['properties'])

bacia_raster =  baciabuffer.reduceToImage(['id_codigo'], ee.Reducer.first()).gt(0)
baciabuffer = baciabuffer.geometry()

know about the geometry 'nunivotto4' >>  7616 loaded < 1 > geometry
{'Shape_Area': 14213.2860311, 'Shape_Leng': 6.80159855416, 'id_codigo': 1, 'nunivotto3': '7616', 'nunivotto4': '7616'}


In [191]:
date_inic = ee.Date.fromYMD(int(nyear),1,1)
date_end = ee.Date.fromYMD(int(nyear),12,31)
# de mosaico EE y para mosaico Mapbiomas (X)
lstCoef = [0.8425, 0.8957, 0.9097, 0.3188, 0.969, 0.9578]
bandsCoef = ee.Image.constant(lstCoef + lstCoef + lstCoef)
lstIntercept = [106.7546, 115.1553, 239.0688, 1496.4408, 392.3453, 366.57]
bandsIntercept = ee.Image.constant(lstIntercept + lstIntercept + lstIntercept)

colmosaicMapbiomas = (imgMosaic.filter(ee.Filter.eq('year', nyear))
                                .median().updateMask(bacia_raster))
imagens_mosaicoEEv = colmosaicMapbiomas.multiply(bandsCoef).add(bandsIntercept)
imagens_mosaicoEEv = imagens_mosaicoEEv.divide(10000)
print(f" we have {imagens_mosaicoEEv.bandNames().getInfo()} images ")



 we have ['blue_median', 'green_median', 'red_median', 'nir_median', 'swir1_median', 'swir2_median', 'blue_median_wet', 'green_median_wet', 'red_median_wet', 'nir_median_wet', 'swir1_median_wet', 'swir2_median_wet', 'blue_median_dry', 'green_median_dry', 'red_median_dry', 'nir_median_dry', 'swir1_median_dry', 'swir2_median_dry'] images 


In [176]:
#cria o mosaico a partir do mosaico total, cortando pelo poligono da bacia
mosaicColGoogle = imagens_mosaicoEE.filter(ee.Filter.date(date_inic, date_end))
mosaicoBuilded = make_mosaicofromIntervalo(mosaicColGoogle, nyear)
mosaicoBuilded = mosaicoBuilded.updateMask(bacia_raster)
# print(f" we have {mosaicoBuilded.bandNames().getInfo()} images do mosaico mensal do google ")
maskGaps = mosaicoBuilded.unmask(-1000).eq(-1000).updateMask(bacia_raster)
## preenchendo o gap do mosaico do EE pelo mosaico dao mapbiomas
mosaicoBuilded = mosaicoBuilded.unmask(0).where(maskGaps, imagens_mosaicoEEv)
mosaicoBuilded = mosaicoBuilded.updateMask(bacia_raster)
lstAllBandasmos = mosaicoBuilded.bandNames().getInfo()
print(f" temos {len(lstAllBandasmos)} imagenes ")
print(f" we have images in mosaico buiding")
for bnd in range(0, len(lstAllBandasmos) + 1, 5):
    print(lstAllBandasmos[bnd:bnd+5])

 temos 18 imagenes 
 we have images in mosaico buiding
['blue_median', 'green_median', 'red_median', 'nir_median', 'swir1_median']
['swir2_median', 'blue_median_dry', 'green_median_dry', 'red_median_dry', 'nir_median_dry']
['swir1_median_dry', 'swir2_median_dry', 'blue_median_wet', 'green_median_wet', 'red_median_wet']
['nir_median_wet', 'swir1_median_wet', 'swir2_median_wet']


In [ ]:
palettes = [
    "#ffffff","#32a65e","#32a65e","#1f8d49","#7dc975","#04381d","#026975","#000000","#000000","#7a6c00","#ad975a",
    "#519799","#d6bc74","#d89f5c","#FFFFB2","#edde8e","#000000","#000000","#f5b3c8","#C27BA0","#db7093","#ffefc3",
    "#db4d4f","#ffa07a","#d4271e","#db4d4f","#0000FF","#000000","#000000","#ffaa5f","#9c0027","#091077","#fc8114",
    "#2532e4","#93dfe6","#9065d0","#d082de","#000000","#000000","#f5b3c8","#c71585","#f54ca9","#cca0d4","#dbd26b",
    "#807a40","#e04cfa","#d68fe2","#9932cc","#e6ccff","#02d659","#ad5100","#000000","#000000","#000000","#000000",
    "#000000","#000000","#CC66FF","#FF6666","#006400","#8d9e8b","#f5d5d5","#ff69b4","#ebf8b5","#000000","#000000",
    "#91ff36","#7dc975","#e97a7a","#0fffe3"
]

visualizar = {
    'visclassCC': {
            "min": 0,
            "max": 69,
            "palette":  palettes,
            "format": "png"
    },
    'visMosaic': {
        'min': 0.001, 'max': 0.15,
        'bands': ['red', 'green', 'blue']
    },
    'vismosaico2':{
        'min': 10,'max': 2000,
        'bands': ['red_median', 'green_median', 'blue_median']
    },
    'vismosaicoGEE': {
        'min': 0.001, 'max': 0.15,
        'bands': ['red_median', 'green_median', 'blue_median']
    },
    'base': {
        'min': 0,'max': 1,
        'palette': ['#000000', '#ffffff']
    },
    'indexIIA' :{
        'min': -0.5, 'max':1,
        'palette': ["#6a8d73","#f4fdd9","#e4ffe1","#ffe8c2","#f0a868"]
    },
    'indexGLI': {
        'min': -0.5, 'max':1,
        'palette': ["#fae3e3","#f7d4bc","#cfa5b4","#c98bb9","#846b8a"]
    },
}

In [ ]:
import geemap.core as geemap
# Initialize a map object.

In [177]:
layerA = geemap.Map()
# Add the image to the map.
layerA.add_layer(baciabuffer, {'color': 'red'}, 'shp')
layerA.add_layer(bacia_raster, visualizar['base'], 'base')
layerA.add_layer(colmosaicMapbiomas, visualizar['vismosaico2'], 'Mosaico MB Org')
layerA.add_layer(imagens_mosaicoEEv, visualizar['vismosaicoGEE'], 'Mosaico MB')
layerA.add_layer(mosaicoBuilded, visualizar['vismosaicoGEE'], 'Mosaico')
layerA.add_layer(maskGaps, {'min':0, 'max': 1}, 'mask') #
# Set the map center.
# m.centerObject(mosaicMond, 18)
# Display the map (you can call the object directly if it is the final line).
display(layerA)

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [178]:
# calculando todos os indices espectrais
bandas_imports = [
    'afvi_median_wet', 'brightness_median', 'green_median', 'gvmi_median', 'swir2_median',  'wetness_median',
    'dswi5_median', 'red_median', 'ndvi_median_wet', 'rvi_median_wet', 'gvmi_median_wet', 'ratio_median_wet',
    'ri_median', 'rvi_median', 'gli_median_wet', 'wetness_median_wet', 'ui_median_wet', 'lswi_median',
    'ndwi_median','ndwi_median_wet', 'awei_median', 'swir1_median_wet', 'awei_median_wet', 'gemi_median_wet',
    'iia_median_dry', 'gcvi_median_dry', 'cvi_median_wet', 'gemi_median', 'osavi_median_wet', 'blue_median',
    'nddi_median_wet', 'bsi_median', 'awei_median_dry', 'shape_median_dry', 'swir2_median_wet', 'nir_median_dry',
    'afvi_median', 'gcvi_median', 'nir_median_wet', 'ndvi_median', 'swir2_median_dry', 'red_median_dry', 'ratio_median',
    'osavi_median', 'avi_median_wet', 'evi_median','swir1_median','nir_median', 'iia_median'
]
mosaicProcess = CalculateIndice(mosaicoBuilded, bandas_imports)
lstBandasBuildMoisac = mosaicProcess.bandNames().getInfo()
for bnd in range(0, len(lstBandasBuildMoisac) + 1, 5):
    print(lstBandasBuildMoisac[bnd:bnd+5])

['blue_median', 'green_median', 'red_median', 'nir_median', 'swir1_median']
['swir2_median', 'blue_median_dry', 'green_median_dry', 'red_median_dry', 'nir_median_dry']
['swir1_median_dry', 'swir2_median_dry', 'blue_median_wet', 'green_median_wet', 'red_median_wet']
['nir_median_wet', 'swir1_median_wet', 'swir2_median_wet', 'nir_median_contrast', 'red_median_contrast']
['nir_median_dry_contrast', 'red_median_dry_contrast', 'afvi_median_wet', 'brightness_median', 'gvmi_median']
['wetness_median', 'dswi5_median', 'ndvi_median_wet', 'rvi_median_wet', 'gvmi_median_wet']
['ratio_median_wet', 'ri_median', 'rvi_median', 'gli_median_wet', 'wetness_median_wet']
['ui_median_wet', 'lswi_median', 'ndwi_median', 'ndwi_median_wet', 'awei_median']
['awei_median_wet', 'gemi_median_wet', 'iia_median_dry', 'gcvi_median_dry', 'cvi_median_wet']
['gemi_median', 'osavi_median_wet', 'nddi_median_wet', 'bsi_median', 'awei_median_dry']
['shape_median_dry', 'afvi_median', 'gcvi_median', 'ndvi_median', 'ratio_med

In [ ]:
layerB = geemap.Map()
# Add the image to the map.
layerB.add_layer(bacia_raster, visualizar['base'], 'base')
layerB.add_layer(mosaicProcess, visualizar['vismosaicoGEE'], 'Mosaico')
layerB.add_layer(mosaicProcess.select('ndwi_median'), visualizar['indexIIA'], 'ndwi_median')
layerB.add_layer(mosaicProcess.select('osavi_median'), visualizar['indexGLI'], 'osavi_median' )
# Set the map center.
# m.centerObject(mosaicMond, 18)
# Display the map (you can call the object directly if it is the final line).
display(layerB)

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [179]:
nameFeatROIs =  f"{nbacia}_{nyear}_cd"
print("loading Rois with name =>>>>>> ", nameFeatROIs)
asset_rois = options['asset_joinsGrBa']
# if not process_mosaic_EE:
#     asset_rois = self.options['asset_joinsGrBaMB']

dir_asset_rois = os.path.join(asset_rois, nameFeatROIs)
print(f"load samples from idAsset >> {dir_asset_rois}")
### carregando a FeatureCollection desde o asset
ROIs_toTrain = ee.FeatureCollection(dir_asset_rois)
print("show size ", ROIs_toTrain.size().getInfo())
### listando todas as bandas
lstPropROIs = ROIs_toTrain.first().propertyNames().getInfo()
print(f"bandas de ROIs {len(lstPropROIs)} \n >>> ")
# for bnd in range(0, len(lstPropROIs) + 1, 5):
#     print(lstPropROIs[bnd:bnd+5])

loading Rois with name =>>>>>>  7616_2018_cd
load samples from idAsset >> projects/mapbiomas-workspace/AMOSTRAS/col10/CAATINGA/ROIs/ROIs_cleaned_downsamplesv4C/7616_2018_cd
show size  28750
bandas de ROIs 146 
 >>> 


In [180]:
print(ROIs_toTrain.aggregate_histogram('class').getInfo())

{'15.0': 7347, '18.0': 2206, '21.0': 4518, '22.0': 889, '29.0': 45, '3.0': 2347, '33.0': 1427, '4.0': 9971}


In [181]:
ROIs_toTrain = ROIs_toTrain.filter(ee.Filter.notNull(bandas_imports))
ROIs_toTrain = ROIs_toTrain.map(lambda f: f.set('class', ee.Number.parse(f.get('class')).toFloat().toInt8()))
print(ROIs_toTrain.aggregate_histogram('class').getInfo())

{'15': 7347, '18': 2206, '21': 4518, '22': 889, '29': 45, '3': 2347, '33': 1427, '4': 9971}


In [182]:
print("show size ", ROIs_toTrain.size().getInfo())

show size  28750


In [183]:
def down_samples_ROIs(rois_train):
    dictROIs = ee.FeatureCollection(rois_train).aggregate_histogram('class').getInfo()
    # print(dictROIs)
    # dictQtLimit = {
    #     '3': 800,
    #     '4': 1800,
    #     '12': 600,
    #     '15': 1200,
    #     '18': 800,
    #     '21': 1000,
    #     '22': 800,
    #     '29': 400,
    #     '33': 400
    # }
    dictQtLimit = {
        '3': 700,
        '4': 1400,
        '12': 500,
        '15': 1000,
        '18': 700,
        '21': 900,
        '22': 600,
        '29': 300,
        '33': 350
    }
    def make_random_select(featCC, limiar):
        featCC = featCC.randomColumn()
        featCC = featCC.filter(ee.Filter.lt('random', ee.Number(limiar).toFloat()))  #float(dictQtLimit[cclass]/sizeFC)
        return featCC

    lstFeats = ee.FeatureCollection([])
    # newNN = 2000
    for cclass in list(dictROIs.keys()):
        print(" filtering class >> ", cclass)
        feattmp = rois_train.filter(ee.Filter.eq('class', int(cclass)))
        sizeFC = feattmp.size()#.getInfo()
        # print("size original ", sizeFC)
        feattmp = ee.Algorithms.If(
                        ee.Algorithms.IsEqual(ee.Number(sizeFC).gt(ee.Number(dictQtLimit[str(cclass)])),1),
                        make_random_select(feattmp, ee.Number(dictQtLimit[str(cclass)]).divide(ee.Number(sizeFC))),
                        feattmp)
        lstFeats = lstFeats.merge(feattmp)
    return ee.FeatureCollection(lstFeats)
ROIs_toTrain = down_samples_ROIs(ROIs_toTrain)

 filtering class >>  15
 filtering class >>  18
 filtering class >>  21
 filtering class >>  22
 filtering class >>  29
 filtering class >>  3
 filtering class >>  33
 filtering class >>  4


In [184]:
dictROIs = ROIs_toTrain.aggregate_histogram('class').getInfo()
print(dictROIs)

{'15': 950, '18': 686, '21': 898, '22': 612, '29': 45, '3': 709, '33': 327, '4': 1382}


In [185]:
pmtroClass = copy.deepcopy(options['pmtGTB'])
pmtroClass['numberOfTrees'] = 18
pmtroClass['shrinkage'] = 0.1    #
pmtroClass['samplingRate'] = 0.55
print(pmtroClass)
makeRF = False

{'numberOfTrees': 18, 'shrinkage': 0.1, 'samplingRate': 0.55, 'loss': 'LeastSquares', 'seed': 0}


In [186]:
bandActiva = f'classification_{nyear}'
# ee.Classifier.smileGradientTreeBoost(numberOfTrees, shrinkage, samplingRate, maxNodes, loss, seed)
classifierGTB = ee.Classifier.smileGradientTreeBoost(**pmtroClass).train(
                                    ROIs_toTrain, 'class', bandas_imports)
classifiedGTB = mosaicProcess.classify(classifierGTB, bandActiva)

In [ ]:
classifiedRF = None
if makeRF:
    pmtRF = {
            'numberOfTrees': 165,
            'variablesPerSplit': 15,
            'minLeafPopulation': 40,
            'bagFraction': 0.8,
            'seed': 0
        }
    classifierRF = ee.Classifier.smileRandomForest(**pmtRF).train(
                                                    ROIs_toTrain, 'class', bandas_imports)
    classifiedRF = mosaicProcess.classify(classifierRF, bandActiva)

In [187]:
layerC = geemap.Map()
# Add the image to the map.
layerC.add_layer(mosaicProcess, visualizar['vismosaicoGEE'], 'Mosaico')
if makeRF:
    layerC.add_layer(classifiedRF, visualizar['visclassCC'], 'mapa RF')
layerC.add_layer(classifiedGTB, visualizar['visclassCC'], 'mapa GTB')
# Display the map (you can call the object directly if it is the final line).
display(layerC)

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [ ]:
def processoExportar(mapaRF, regionB, nameB):
    nomeDesc = 'BACIA_'+ str(nameB)
    idasset =  os.path.join(options['assetOut'] , nomeDesc)
    optExp = {
        'image': mapaRF,
        'description': nomeDesc,
        'assetId':idasset,
        'region':ee.Geometry(regionB), #['coordinates'] .getInfo()
        'scale': 30,
        'maxPixels': 1e13,
        "pyramidingPolicy":{".default": "mode"},
        # 'priority': 1000
    }
    task = ee.batch.Export.image.toAsset(**optExp)
    task.start()
    print("salvando ... " + nomeDesc + "..!")
    # print(task.status())
    for keys, vals in dict(task.status()).items():
        print ( "  {} : {}".format(keys, vals))

In [188]:
nomec = f"{nbacia}_{nyear}_GTB_col10-v_{options['version']}"
print("salvando >> ", nomec)
mydict = {
    'id_bacia': nbacia,
    'version': options['version'],
    'biome': options['bioma'],
    'classifier': 'GTB',
    'collection': '10.0',
    'sensor': 'Landsat',
    'source': 'geodatin',
    'year': nyear
}
classifiedGTB = classifiedGTB.set(mydict)
classifiedGTB = classifiedGTB.set("system:footprint", baciabuffer.coordinates())
processoExportar(classifiedGTB, baciabuffer, nomec)

salvando >>  7616_2018_GTB_col10-v_4
salvando ... BACIA_7616_2018_GTB_col10-v_4..!
  state : READY
  description : BACIA_7616_2018_GTB_col10-v_4
  priority : 100
  creation_timestamp_ms : 1746398113965
  update_timestamp_ms : 1746398113965
  start_timestamp_ms : 0
  task_type : EXPORT_IMAGE
  id : 5ELCC5UEBFRFOQJ3ARCBCKS5
  name : projects/geo-data-s/operations/5ELCC5UEBFRFOQJ3ARCBCKS5
